In [1]:
import pandas as pd
import scripts.data_pull as dp
import numpy as np
from sklearn.feature_extraction import text
from joblib import parallel_backend

https://stackoverflow.com/questions/26826002/adding-words-to-stop-words-list-in-tfidfvectorizer-in-sklearn

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
porter_stemmer = PorterStemmer()
project_stopwords = stopwords.words("english")
corpus_specific_stopwords = ['000', '10']
project_stopwords.extend(corpus_specific_stopwords)

def nltk_tokenizer(str_input):
    words = word_tokenize(str_input)
    words = porter_stemmer.stem(words)
    return words

In [3]:
full_corpus = dp.data_pull("combined_corpus")
# Found by Initial Analysis
corpus_specific_stopwords = text.ENGLISH_STOP_WORDS.union(["000", "19", "10"])

NameError: name 'dp' is not defined

In [4]:
x_text = full_corpus.text
y_bias = full_corpus.art_bias

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_text, y_bias,
    test_size=.2,
    random_state=42
)

NameError: name 'x_text' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=nltk_tokenizer,
                             max_features=200)),
    ('clf', LogisticRegression()),
])

In [ ]:
with parallel_backend('threading', n_jobs = 5):
    text_clf.fit(x_train, y_train)

In [ ]:
predicted = text_clf.predict(x_test)
np.mean(predicted == y_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

In [ ]:
metrics.confusion_matrix(y_test, predicted)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'vect__max_features': (None, 200)
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf.fit(x_train, y_train)

In [ ]:
gs_predicted = gs_clf.predict(x_test)
np.mean(gs_predicted == y_test)

In [ ]:
gs_results = pd.DataFrame(gs_clf.cv_results_)

In [ ]:
gs_results

In [ ]:
metrics.confusion_matrix(y_test, gs_predicted)

In [ ]:
coef_values = pd.DataFrame(text_clf[1].coef_,columns=text_clf[0].get_feature_names_out())

In [ ]:
coef_values.head()

In [ ]:
text_clf[0].get_params()